Spoken and visual twist on the original Mad Lib game.

Use Google text-to-speech and speech-to-text, GPT-4, and Dall-E.

Ex: python glibgab.py -l 1 -w 5 -i "da Vinci's painting of the Mona Lisa"

Copyright 2023, Arno Klein, MIT License

In [1]:
from generate_text_images import generate_description, generate_madlib
from generate_text_images import fill_madlib, fix_grammar
from generate_text_images import generate_image
from io_files import display_image

In [ ]:
    >>> specific_instance = "Leonardo Da Vinci's painting The Last Supper"
    >>> max_reading_level = 9
    >>> topic = "famous artwork"
    >>> verbose = True
    >>> generate_description(topic, specific_instance, max_reading_level, verbose)

In [ ]:
    >>> description = "Sam's friends are playing."
    >>> select_max_words = 5
    >>> verbose = True
    >>> madlib, unique_words = generate_madlib(description, select_max_words, verbose)

In [ ]:
    >>> madlib = "I ate one gala {0}, three fuji {0}s, and one {1}'s flesh."
    >>> words = ['apple', 'cantaloupe']
    >>> response_duration = 2
    >>> verbose = True
    >>> filled_madlib, new_words = fill_madlib(madlib, words, response_duration, verbose)

In [ ]:
    >>> input_text = "I ate one gala strawberry, three fuji strawberrys, and one honeydew's flesh."
    >>> verbose = True
    >>> fix_grammar(input_text, verbose)

In [2]:
    >>> prompt = "da Vinci's painting of the Mona Lisa"
    >>> size = "256x256"
    >>> output_format = "png"
    >>> output_dir = 'tmp'
    >>> verbose = True
    >>> png_file = generate_image(prompt, size, output_format, output_dir, verbose)

tmp/da Vi-1693369541-0.png


In [3]:
display_image(png_file)